In [1]:
from pipeline import segmentationPipeline
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from utils import displayDice, dice_per_class
from imageviewer import showSegmentation

In [3]:
pipeline = segmentationPipeline(device='cuda:0')

In [8]:
root = '/datadrive/redoFinetuning/exhale_rotated'

images = sorted([val for val in os.listdir('/datadrive/redoFinetuning/exhale_rotated') if 'image' in val])
masks = sorted([val for val in os.listdir('/datadrive/redoFinetuning/exhale_rotated') if 'mask' in val])

In [3]:
root = '/datadrive/redoFinetuning/inhale_rotated'
images = sorted([val for val in os.listdir(root) if 'image' in val])
masks = sorted([val for val in os.listdir(root) if 'mask' in val])

In [9]:
root

'/datadrive/redoFinetuning/exhale_rotated'

In [10]:
for i,file in enumerate(images):
    img = np.load(os.path.join(root,file))
    img = img / 255
    img = np.clip(img,0,1)
    np.save(os.path.join(root,file),img)
    # mask = np.load(os.path.join(root,masks[i]))
    # out = pipeline.segment(img)
    # #showSegmentation(img,out)
    # displayDice(dice_per_class(torch.tensor(mask).cuda(),torch.tensor(out).unsqueeze(0).unsqueeze(0).cuda(),num_classes=6))
    #np.save(file.replace('.npy','_seg21524.npy'),out.squeeze(0).squeeze(0).cpu().numpy())
# viewer = napari.view_image(mask,name='gt',colormap='gist_earth',contrast_limits=(0,5))
# viewer.add_image(out.squeeze(0).squeeze(0).cpu().numpy(),colormap='gist_earth',name='segmentation',contrast_limits=(0,5))

In [4]:
for i,file in enumerate(images):
    print(file,masks[i])
    img = np.load(os.path.join(root,file))
    img = img / 255
    img = np.clip(img,0,1)
    mask = np.load(os.path.join(root,masks[i]))
    out = pipeline.segment(img)
    showSegmentation(img,out)
    break
    #displayDice(dice_per_class(torch.tensor(mask).cuda(),torch.tensor(out).unsqueeze(0).unsqueeze(0).cuda(),num_classes=6))
    #np.save(file.replace('.npy','_seg21524.npy'),out.squeeze(0).squeeze(0).cpu().numpy())
# viewer = napari.view_image(mask,name='gt',colormap='gist_earth',contrast_limits=(0,5))
# viewer.add_image(out.squeeze(0).squeeze(0).cpu().numpy(),colormap='gist_earth',name='segmentation',contrast_limits=(0,5))

B10_00.mat_inhale_image.nii.npy B10_00_inhale_mask_final.nii.gz.npy


In [5]:
from scipy.io import loadmat,savemat

for file in os.listdir('./source'):
    workspace = loadmat(os.path.join('./source',file))
    img = workspace['spectCT']
    img = img / 1000
    img = np.clip(img,0,1)
    img = np.rot90(img, k=1, axes=(0,1))
    img = np.rot90(img, k=1, axes=(0,2))
    img = np.flip(img, axis=2)
    out = pipeline.segment(img)
    workspace['spectCT_mask'] = out
    showSegmentation(img,out)
    break
    savemat(os.path.join('./dest',file),workspace) 
    


In [8]:
ganoutput = np.load('/home/aaronluong/Downloads/testGANoutput.npy')
ganoutput = np.clip(ganoutput,0,1/1.624)
ganoutput = ganoutput * 1.624
ganoutput = np.flip(ganoutput,1)


showSegmentation(ganoutput,pipeline.segment(ganoutput,getLR=True))